# Not using for KN

# Convert MAGS (Vega or AB mag system) to Flux
You can use the customized package what_the_flux (written by Chris Frohmaier, with a small contribution from Maria Vincenzi).

Here you can find a quick example.

Remember that:
1) 90% of the times Core Collapse SNe are presented in the standard filter system (not in natural filter system). This means that photometry is presented as it would have been observed by the "standard" Bessell/SDSS filter set. The "standard" Bessell/SDSS filter sets are stored in inputs/Filters/GeneralFilters. The main exception is CSP data, usually presented in natural system. Therfore, for CSP objects we use inputs/Filters/Site3_CSP.

2) Magnitudes in Bessell filters (UBVRI) are usually calculated in the Vega system and calibrated to Landolt standards

3) Magnitudes in SDSS filters (ugriz) are usually calculated in the AB system and calibrated to Smith 2002 standards

4) Magnitudes in Space Telescope SWIFT filters (W2,M2,W1,U,B,V) are usually calculated in the Vega system. See Peter Brown's paper for calibration of the instrument.


In [4]:
import os
COCO_PATH="/Users/ravkaur/Desktop/research/kilonova-SED/PyCoCo_templates/"
#COCO_PATH=os.environ['COCO_PATH']

In [6]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, COCO_PATH+'what_the_flux/')
import what_the_flux as wtf

In [7]:
snname = 'SN1993J'

In [8]:
INPUT_LC_PATH = COCO_PATH+"Inputs/Photometry/0_LCs_mags_raw/%s_mag.dat"%snname
OUTPUT_LC_PATH = COCO_PATH+"Inputs/Photometry/1_LCs_flux_raw/%s.dat"%snname
FILTERS_PATH = COCO_PATH+"Inputs/Filters/"

In [9]:
Vega_filt = ['Bessell_U', 'Bessell_B','Bessell_V','Bessell_R','Bessell_I', 
            'swift_UVW2', 'swift_UVM2', 'swift_UVW1', 'swift_U', 'swift_B','swift_V',
            'Y', 'H', 'J', 'K', 'Ks']
AB_filt = ['sdss_u', 'sdss_g','sdss_r','sdss_i','sdss_z', 
            "sdss_u'", "sdss_g'", "sdss_r'", "sdss_i'", "sdss_z'"]

CSP_SNe = ['SN2004fe', 'SN2005bf', 'SN2006V', 'SN2007C', 'SN2007Y',
           'SN2009bb',  'SN2008aq', 'SN2006T', 'SN2004gq', 'SN2004gt',
           'SN2004gv','SN2006ep', 'SN2008fq', 'SN2006aa']


In [11]:
fout = open(OUTPUT_LC_PATH, 'w')
fout.write('# From Mags to Flux, not MW/Host dust corrected\n')
fout.close()

phot_mags= pd.read_csv(INPUT_LC_PATH, comment='#')
filter_set = 'GeneralFilters' if snname not in CSP_SNe else 'Site3_CSP'
                
fluxes = []
fluxes_err = []
filter_set_list = []             

for index, row in phot_mags.iterrows():
    
    w,t = wtf.loadFilter(FILTERS_PATH+'%s/%s.dat'%(filter_set, row['band']))
    if row['band'] in Vega_filt: band = wtf.Band_Vega(w,t)
    elif row['band'] in AB_filt: band = wtf.Band_AB(w,t)
    else: print ('Filter system?!?!?')
    flux_withMW = wtf.mag2flux(band, row['Mag']).value
    flux_err_withMW = wtf.ERRmag2ERRflux(band, row['Mag'], row['Mag_err']).value
    fluxes.append(flux_withMW)
    fluxes_err.append(flux_err_withMW)
    filter_set_list.append(filter_set)
                
phot_mags['Flux'] = fluxes
phot_mags['Flux_err'] = fluxes_err
phot_mags['FilterSet'] = filter_set_list
phot_mags[['MJD','Mag','Mag_err','Flux','Flux_err','band','Instr','FilterSet']].to_csv(OUTPUT_LC_PATH, index=False)

UnitConversionError: '' (dimensionless) and 'Angstrom' (length) are not convertible